In [ ]:
!pip install mediawikiapi
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 3.1 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully 

In [ ]:
from mediawikiapi import MediaWikiAPI
import pandas as pd
import tqdm
import datetime as dt
import asyncio
import multiprocessing
import os
import psutil
import threading
import time
import concurrent.futures
from mediawikiapi.exceptions import PageError
import random
from requests.exceptions import ConnectionError
from mediawikiapi.exceptions import MediaWikiAPIException
#from requests import JSONDecodeError
from json import JSONDecodeError

import os
import datasets 
import numpy as np
mediawikiapi = MediaWikiAPI()


mediawikiapi.config.language = "no"
dataset_nowiki = datasets.load_dataset("jkorsvik/nowiki_abstract_urls_20230120")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/605457 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/jkorsvik___parquet/jkorsvik--nowiki_abstract_urls_20230120-1591f4975dea9ad4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset_nowiki["train"]

Dataset({
    features: ['title', 'url', 'abstract', 'id'],
    num_rows: 605457
})

In [ ]:
nowiki_df = pd.DataFrame(dataset_nowiki["train"])
nowiki_df["title"] = nowiki_df["title"].apply(
    lambda x: x.replace("Wikipedia: ", ""))



In [ ]:
!pip install huggingface_hub
!git config --global credential.helper store
!huggingface-cli login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add tok

In [ ]:
SPLIT = 1
# nowiki_articles = nowiki_df["title"]  # .values[:100]
df_split = np.array_split(nowiki_df, 3)
no_wiki_split = df_split[SPLIT-1]
urls = []
headlines = []
articles = []
ingresses = []
categories = []
abstracts = []

# Split the list of URLs into batches

batch_size = 10  # os.cpu_count() - 2
num_cores = os.cpu_count() - 2


def collect_article_data(name, depth=0):
    try:
        page = mediawikiapi.page(name)
        title = page.title
        url = page.url
        content = page.content

        categories = page.categories
        summary = page.summary
        # Some if-test to check if the page is a redirected page
        # and if there is a valid hit
        return title, url, content, summary, categories
    except JSONDecodeError as e:
        print("Got JSONDecodeError, skipping")
        print(f"Error fetching the article: {name}")
        return name, None, None, None, None

        time.sleep(random.random()*10)
        if depth < 5:
            return collect_article_data(name, depth=depth+1)
        else:
            print("Got JSONDecodeError, skipping")
            print(f"Error fetching the article: {name}")
            return name, None, None, None, None

    except KeyError as e:
        print("Got KeyError, skipping")

        print(f"Error fetching the article: {name}")
        return name, None, None, None, None

    except PageError as e:
        print("Got PageErrors, skipping")

        print(f"Error fetching the article: {name}")
        return name, None, None, None, None

    except ConnectionError as e:
        print("Got PageErrors, skipping")

        print(f"Error fetching the article: {name}")
        return name, None, None, None, None
    except MediaWikiAPIException as e:
        print("Got PageErrors, skipping")

        print(f"Error fetching the article: {name}")
        return name, None, None, None, None


def main():
    # Split the list of URLs into batches
    wiki_name_batches = [no_wiki_split.iloc[i:i + batch_size][["title", "url", "abstract"]]
                         for i in range(0, len(no_wiki_split), batch_size)]
    # print(wiki_name_batches)

    for url_batch in tqdm.tqdm(wiki_name_batches):

        with concurrent.futures.ThreadPoolExecutor(max_workers=batch_size) as executor:
            res = executor.map(collect_article_data,
                               url_batch["title"].values)
            i = 0
            for result in res:
                
                title, url, content, summary, category = result
                urls.append(url)
                articles.append(content)
                ingresses.append(summary)
                categories.append(category)
                headlines.append(title)
                abstracts.append(url_batch.iloc[i].abstract)
                i += 1
                
        """urls.append(url)
        articles.append(content)
        ingresses.append(summary)
        categories.append(category)
        headlines.append(title)
        abstracts.append(url_batch.abstract.values)
        """

        

    date = dt.datetime.now().strftime("%Y-%m-%d")

    dates = [date for _ in range(len(urls))]
    print(len(abstracts), print(len(urls)))

    nowiki_collected_df = pd.DataFrame({"url": urls, "date_scraped": dates, "headline": headlines,
                                        "category": categories, "ingress": ingresses, "article": articles, "abstract": abstracts})

    print(nowiki_collected_df.shape)
    nowiki_collected_df.to_csv(
        "nowiki_collection_abstracts.csv", index_label="id")
    

if __name__ == "__main__":
    main()


  0%|          | 13/20182 [00:06<2:41:22,  2.08it/s]

Got KeyError, skipping
Error fetching the article: Departement


  0%|          | 31/20182 [00:15<2:39:11,  2.11it/s]

Got PageErrors, skipping
Error fetching the article: Kopper


  0%|          | 50/20182 [00:24<2:45:52,  2.02it/s]

...

 39%|███▉      | 7896/20182 [1:46:24<3:03:19,  1.12it/s]

Got KeyError, skipping
Error fetching the article: .im


 39%|███▉      | 7897/20182 [1:46:25<2:49:31,  1.21it/s]

Got KeyError, skipping
Error fetching the article: .cg


 40%|███▉      | 7993/20182 [1:48:10<4:24:23,  1.30s/it]

# Ny del

In [ ]:
df = pd.read_csv("nowiki_collection_abstracts.csv", index_col=0)
dataset = datasets.Dataset.from_pandas(df)
dataset.push_to_hub(f"jkorsvik/nowiki_abstract_second_scrape_split{SPLIT}")